# Model Comparison for Count Data with Excess Zeros

## Common assumptions (all models)

1. **Independence**: Observations $Y_i$ are conditionally independent given covariates.
    
2. **Systematic component (count part)**:
    $$
    \log(\mu_i) = \beta_0 + \beta_1 X_{i1} + \cdots + \beta_k X_{ik}
    $$
    
3. **Inflation component (when applicable)**:
    Logit or multinomial-logit link for mixing probabilities.

---

## 1. Poisson Regression

**Distribution**
$$
Y_i \sim \text{Poisson}(\mu_i)
$$

**Mean--Variance**
$$
E(Y_i)=\mu_i,\quad \text{Var}(Y_i)=\mu_i
$$

**PMF**
$$
P(Y_i=y)=\frac{\mu_i^y e^{-\mu_i}}{y!}
$$

**Log-likelihood**
$$
\ell(\beta)=\sum_{i=1}^n \left[ y_i \log \mu_i - \mu_i - \log(y_i!) \right]
$$

**Link function**
$$
\log(\mu_i)=X_i^\top\beta
$$

**Estimation**
- **IRLS (Fisher scoring)**
- Standard GLM (Nelder & Wedderburn, 1972)

---

## 2. Negative Binomial (NB2)

**Distribution**
$$
Y_i \sim \text{NB}(\mu_i,\alpha)
$$

**Mean--Variance**
$$
E(Y_i)=\mu_i,\quad \text{Var}(Y_i)=\mu_i+\alpha\mu_i^2
$$

**PMF**
$$
P(Y_i=y)=\frac{\Gamma(y+\alpha^{-1})}{\Gamma(\alpha^{-1})\,y!}
\left(\frac{1}{1+\alpha\mu_i}\right)^{\alpha^{-1}}
\left(\frac{\alpha\mu_i}{1+\alpha\mu_i}\right)^y
$$

**Link**
$$
\log(\mu_i)=X_i^\top\beta
$$

**Log-likelihood**
$$
\ell(\beta,\alpha)=\sum_{i=1}^n \log P(Y_i=y_i)
$$

**Estimation**
- **IRLS + profile likelihood** for $\alpha$
- Not a pure exponential-family GLM

---

## 3. Zero-Inflated Poisson (ZIP)

**Latent structure**
$$
Y_i=
\begin{cases}
0 & \text{with prob } \pi_i \\
\text{Poisson}(\mu_i) & \text{with prob } 1-\pi_i
\end{cases}
$$

**PMF**
$$
P(Y_i=0)=\pi_i+(1-\pi_i)e^{-\mu_i}
$$
$$
P(Y_i=y)= (1-\pi_i)\frac{\mu_i^y e^{-\mu_i}}{y!},\quad y\ge1
$$

**Mean--Variance**
$$
E(Y_i)=(1-\pi_i)\mu_i
$$
$$
\text{Var}(Y_i)=(1-\pi_i)\mu_i(1+\pi_i\mu_i)
$$

**Links**
$$
\log(\mu_i)=X_i^\top\beta
$$
$$
\text{logit}(\pi_i)=Z_i^\top\gamma
$$

**Log-likelihood**
$$
\ell(\beta,\gamma)=\sum_{i=1}^n \log P(Y_i=y_i)
$$

**Estimation**
- **EM algorithm** (Lambert, 1992)
- M-step uses **IRLS** for Poisson and logistic submodels

---

## 4. Zero-Inflated Negative Binomial (ZINB)

**Structure**
$$
Y_i=
\begin{cases}
0 & \text{with prob } \pi_i \\
\text{NB}(\mu_i,\alpha) & \text{with prob } 1-\pi_i
\end{cases}
$$

**PMF**
$$
P(Y_i=0)=\pi_i+(1-\pi_i)P_{\text{NB}}(0)
$$
$$
P(Y_i=y)=(1-\pi_i)P_{\text{NB}}(y),\quad y\ge1
$$

**Mean--Variance**
$$
E(Y_i)=(1-\pi_i)\mu_i
$$
(overdispersion from both inflation and NB)

**Links**
$$
\log(\mu_i)=X_i^\top\beta
$$
$$
\text{logit}(\pi_i)=Z_i^\top\gamma
$$

**Estimation**
- **EM algorithm**
- NB parameters via numerical maximization

---

## 5. Zero-and-(k)-Inflated Poisson (ZkIP)

**Latent mixture**
$$
Y_i=
\begin{cases}
0 & \text{with prob } \pi_{1i} \\
k & \text{with prob } \pi_{2i} \\
\text{Poisson}(\mu_i) & \text{with prob } \pi_{3i}=1-\pi_{1i}-\pi_{2i}
\end{cases}
$$

**PMF**
$$
P(Y_i=0)=\pi_{1i}+\pi_{3i}e^{-\mu_i}
$$
$$
P(Y_i=k)=\pi_{2i}+\pi_{3i}\frac{\mu_i^k e^{-\mu_i}}{k!}
$$
$$
P(Y_i=y)=\pi_{3i}\frac{\mu_i^y e^{-\mu_i}}{y!},\quad y\neq0,k
$$

**Mean--Variance**
$$
E(Y_i)=k\pi_{2i}+\pi_{3i}\mu_i
$$

**Links**
$$
\log(\mu_i)=X_i^\top\beta
$$
$$
\log\frac{\pi_{1i}}{\pi_{3i}}=\gamma_1,\quad
\log\frac{\pi_{2i}}{\pi_{3i}}=\gamma_2
$$

**Estimation**
- **EM algorithm**
- Multinomial-logit (inflation) + Poisson regression (Arora & Chaganty, 2021)

---

## 6. Zero-and-(k)-Inflated Negative Binomial (ZkINB)

**Structure**
$$
Y_i=
\begin{cases}
0 & \pi_{1i} \\
k & \pi_{2i} \\
\text{NB}(\mu_i,\alpha) & \pi_{3i}
\end{cases}
$$

**Mean**
$$
E(Y_i)=k\pi_{2i}+\pi_{3i}\mu_i
$$

**Variance**
- From mixture + NB overdispersion (no closed simple form)

**Links**
$$
\log(\mu_i)=X_i^\top\beta
$$
$$
\text{multinomial-logit for } (\pi_{1i},\pi_{2i},\pi_{3i})
$$

**Estimation**
- **EM algorithm**
- NB likelihood in M-step (numerical)

---

## Key Hierarchy Insight

| Model   | Handles             | Why                            |
|---------|---------------------|--------------------------------|
| Poisson | Equi-dispersion     | Baseline                       |
| NB      | Overdispersion      | Gamma–Poisson mixture          |
| ZIP     | Excess zeros        | Structural zeros               |
| ZINB    | Zeros + heavy tails | Two sources of overdispersion  |
| ZkIP    | Spikes at 0 and k   | Questionnaire / design effects |
| ZkINB   | Spikes + fat tails  | Most flexible                  |